# Extract External catalogs

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/29
- update : 2024/02/29


Access to the external catalog

``quantum graphs ``: https://tigress-web.princeton.edu/~lkelvin/pipelines/w_2024_07/drp_pipe/LATISS/DRP/pipeline_drp_pipe_LATISS_DRP_step1.pdf

In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_07/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_07/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_07/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+34187316c4 	current w_2024_07 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
#import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle

In [5]:
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import coordinates

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [7]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [8]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [9]:
transform = AsinhStretch() + PercentileInterval(99.)

In [10]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

## Configuration

In [11]:
DATE = 202402
#filesource_out="finalized_src_table_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [12]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry
skymap_auxtel ='latiss_v1'

In [13]:
#file_out="sourceTable_202312.csv"

## Give the collection

In [14]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

## Config

# Tracts and Patches

In [15]:
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=my_collection)

In [16]:
with open('../data/202402/Latiss_tractspatchs_202402.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

In [17]:
tracts_patchs_dict

{10644: array([189, 183, 184, 164, 185, 203, 201, 207, 182, 162, 166, 165, 181,
        161, 188, 208, 180, 202]),
 5634: array([ 90, 110,  71, 112,  73, 109,  93,  92,  91,  53,  72,  55,  52,
        111,  54,  74]),
 3863: array([223, 261, 241, 242, 262, 260, 243, 222, 224]),
 452: array([41, 62, 22, 25, 23, 44, 42, 63, 43, 24]),
 5614: array([250, 251, 230, 210, 212, 232, 211, 213, 249, 229, 231]),
 5616: array([355, 358, 316, 338, 356, 336, 357, 337, 317, 335, 315, 334]),
 5615: array([274, 331, 312, 332, 294, 295, 293, 313, 333, 314, 275, 276]),
 5839: array([9, 8, 7]),
 10643: array([190, 171]),
 455: array([51, 52, 15, 33, 14, 53, 34, 13, 32]),
 382: array([343, 344, 342]),
 3864: array([274, 238, 217, 218, 273, 255, 235, 275, 237, 257, 219, 236, 256,
        254, 276, 216]),
 383: array([359, 360])}

In [18]:
tract_selected = 3864
suptitle = f" tract {tract_selected}"
tractInfo = skymap.generateTract(tract_selected)

In [19]:
tractInfo.contains 

<bound method TractInfo.contains of TractInfo(id=3864, ctrCoord=[-0.431459839433178, 0.6789142860803891, -0.5940688504813662])>

In [20]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [21]:
def get_bbox_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    ramin,ramax,decmin,decmax in decimal degrees
    """

    xmin = bbox.beginX
    xmax = bbox.endX
    ymin = bbox.beginY
    ymax = bbox.endY
    
    radec_ll = wcs.pixelToSky(xmin, ymin)
    radec_ur = wcs.pixelToSky(xmax, ymax)
        
    return radec_ur.getRa().asDegrees(),radec_ll.getRa().asDegrees(), radec_ll.getDec().asDegrees(),radec_ur.getDec().asDegrees() 

In [22]:
def isradec_inbbox_radec(ra,dec,wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    Bool
    
    """

    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, bbox)

    assert RAMIN < RAMAX
    assert DECMIN < DECMAX

    flag = (ra >= RAMIN) and (ra<=RAMAX) and (dec>= DECMIN) and (dec <= DECMAX) 
    return flag

In [23]:
def FindTractAndPatch(row):
    """
    Apply this function on  dataframe to find the tract and patch for each visit
    """
    tractsId_list = list(tracts_patchs_dict.keys())   
    try:
        ra = row["ra"]
        dec = row["dec"]
        selectFlag = False
        # loop on tract
        for tractID in tractsId_list:
            wcs = tractsInfoWCS_dict[tractID]
            tractInfo = tractsInfo_dict[tractID] 
            patches_selected = list(tracts_patchs_dict[tractID])
            # loop on patches
            for patch in tractInfo:
                patchID = patch.getSequentialIndex()
                if patchID in patches_selected:
                    ibb=patch.getInnerBBox()
                    flag = isradec_inbbox_radec(ra,dec,wcs, ibb)
                    selectFlag =  selectFlag or flag
                    if selectFlag:
                        return pd.Series([tractID,patchID])
        return pd.Series([0,0])           
                
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        return pd.Series([0,0])  
        

In [24]:
tractsInfo_dict = {}
tractsInfoWCS_dict = {}
tractsId_list = list(tracts_patchs_dict.keys())

# loop to select the good tracts
for tractInfo in skymap:
    tractID = tractInfo.getId()
    # is this tract in Auxtel tract list
    if tractID in tractsId_list:
        # keep tractInfo
        tractsInfo_dict[tractID] = tractInfo 
        # keep WCS
        tractsInfoWCS_dict[tractID] = tractInfo.getWcs()

In [25]:
tractsInfo_dict

{382: TractInfo(id=382, ctrCoord=[0.2774524062533926, 0.04892234508495915, -0.9594929736144974]),
 383: TractInfo(id=383, ctrCoord=[0.27213275275959, 0.0729177513359905, -0.9594929736144974]),
 452: TractInfo(id=452, ctrCoord=[0.30654667281602505, 0.0, -0.9518556284360695]),
 455: TractInfo(id=455, ctrCoord=[0.2976389836299007, 0.07336142064010766, -0.9518556284360695]),
 3863: TractInfo(id=3863, ctrCoord=[-0.4095905537697173, 0.6923277974705078, -0.5940688504813662]),
 3864: TractInfo(id=3864, ctrCoord=[-0.431459839433178, 0.6789142860803891, -0.5940688504813662]),
 5614: TractInfo(id=5614, ctrCoord=[-0.4769266805210716, -0.7745781486588648, -0.4154150130018864]),
 5615: TractInfo(id=5615, ctrCoord=[-0.4548159976772596, -0.7877644160721412, -0.4154150130018864]),
 5616: TractInfo(id=5616, ctrCoord=[-0.43234101369436045, -0.8003196954032654, -0.4154150130018864]),
 5634: TractInfo(id=5634, ctrCoord=[0.012872062229049682, -0.9095409155099149, -0.4154150130018864]),
 5839: TractInfo(id=5

In [26]:
tractsInfoWCS_dict

{382: FITS standard SkyWcs:
 Sky Origin: (10.0000000000, -73.6363636364)
 Pixel Origin: (28499, 28499)
 Pixel Scale: 0.1 arcsec/pixel,
 383: FITS standard SkyWcs:
 Sky Origin: (15.0000000000, -73.6363636364)
 Pixel Origin: (28499, 28499)
 Pixel Scale: 0.1 arcsec/pixel,
 452: FITS standard SkyWcs:
 Sky Origin: (0.0000000000, -72.1487603306)
 Pixel Origin: (28499, 28499)
 Pixel Scale: 0.1 arcsec/pixel,
 455: FITS standard SkyWcs:
 Sky Origin: (13.8461538462, -72.1487603306)
 Pixel Origin: (28499, 28499)
 Pixel Scale: 0.1 arcsec/pixel,
 3863: FITS standard SkyWcs:
 Sky Origin: (120.6091370558, -36.4462809917)
 Pixel Origin: (28499, 28499)
 Pixel Scale: 0.1 arcsec/pixel,
 3864: FITS standard SkyWcs:
 Sky Origin: (122.4365482234, -36.4462809917)
 Pixel Origin: (28499, 28499)
 Pixel Scale: 0.1 arcsec/pixel,
 5614: FITS standard SkyWcs:
 Sky Origin: (238.3783783784, -24.5454545455)
 Pixel Origin: (28499, 28499)
 Pixel Scale: 0.1 arcsec/pixel,
 5615: FITS standard SkyWcs:
 Sky Origin: (240.000

# Reference Sources

In [27]:
#data_product = "gaia_dr2_20200414" 
data_product = "gaia_dr3_20230707"
#data_product = 'atlas_refcat2_20220201'

datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")
filesource_out=f"{data_product}_{DATE}.csv"

In [28]:
all_df = []

for i, ref in enumerate(datasetRefs):
    print(f"========================datasetType = {data_product} ============================================")
    print("fullId.............................:",ref.dataId.full)
  
    skypix = ref.dataId['htm7']
    t = butler.get(ref).asAstropy()
    df = t.to_pandas()
    df.rename(columns={"coord_ra": "ra", "coord_dec": "dec"}, errors="raise",inplace=True)
    df[["tractID","patchID"]] = df.apply( FindTractAndPatch, axis=1,result_type ='expand')
    if(len(df)>0):
        print(type(df),len(df))
        df['skypix'] = skypix
        all_df.append(df)

    if i> 20:
        break
    

========================datasetType = gaia_dr3_20230707 ============================================
fullId.............................: {htm7: 131072}


/tmp/ipykernel_8973/690092218.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


<class 'pandas.core.frame.DataFrame'> 820
========================datasetType = gaia_dr3_20230707 ============================================
fullId.............................: {htm7: 131073}


/tmp/ipykernel_8973/690092218.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


<class 'pandas.core.frame.DataFrame'> 733
========================datasetType = gaia_dr3_20230707 ============================================
fullId.............................: {htm7: 131074}


...

<class 'pandas.core.frame.DataFrame'> 785
========================datasetType = gaia_dr3_20230707 ============================================
fullId.............................: {htm7: 131092}


/tmp/ipykernel_8973/690092218.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


<class 'pandas.core.frame.DataFrame'> 746
========================datasetType = gaia_dr3_20230707 ============================================
fullId.............................: {htm7: 131093}


/tmp/ipykernel_8973/690092218.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


<class 'pandas.core.frame.DataFrame'> 754


In [ ]:
df.columns

In [30]:
df = pd.concat(all_df)

In [31]:
df

,id,ra,dec,phot_g_mean_flux,phot_bp_mean_flux,phot_rp_mean_flux,phot_g_mean_fluxErr,phot_bp_mean_fluxErr,phot_rp_mean_fluxErr,coord_raErr,coord_decErr,epoch,pm_ra,pm_dec,pm_raErr,pm_decErr,pm_flag,parallax,parallaxErr,parallax_flag,coord_ra_coord_dec_Cov,coord_ra_pm_ra_Cov,coord_ra_pm_dec_Cov,coord_ra_parallax_Cov,coord_dec_pm_ra_Cov,coord_dec_pm_dec_Cov,coord_dec_parallax_Cov,pm_ra_pm_dec_Cov,pm_ra_parallax_Cov,pm_dec_parallax_Cov,astrometric_excess_noise,tractID,patchID,skypix
0,2449850960546191232,0.000166,-0.011876,3.968388e+04,1.296045e+04,9.950487e+04,136.475806,1315.772563,2662.131182,2.096146e-09,1.816218e-09,57388.499407,1.467858e-08,-4.274311e-08,2.581790e-09,2.036342e-09,False,8.576050e-09,2.610054e-09,False,1.045344e-18,1.850645e-19,-1.341398e-19,8.106214e-19,1.576844e-19,2.370378e-18,-1.459838e-18,2.034396e-18,-1.022296e-18,-2.190038e-18,1.349154,0,0,131072
1,2449851338503315712,0.000105,-0.011755,1.959243e+05,7.682066e+04,6.016697e+05,1077.335205,2006.439947,3308.406644,3.788678e-09,2.637310e-09,57388.499407,NaN,NaN,NaN,NaN,False,NaN,NaN,False,1.576009e-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.491338,0,0,131072
2,2449851377159675392,0.000333,-0.011656,3.874935e+04,1.853878e+04,8.387241e+04,141.264120,1849.701766,2043.742181,2.157922e-09,1.816136e-09,57388.499407,1.641371e-08,-2.032592e-08,2.640814e-09,1.952416e-09,False,4.068542e-09,2.890777e-09,False,1.020630e-18,-1.382141e-19,-4.374544e-19,5.912648e-19,-3.359493e-19,1.918172e-18,-2.041828e-18,1.715501e-18,-1.513156e-18,-2.908092e-18,1.641737,0,0,131072
3,2449851407223073792,0.000664,-0.011549,1.726028e+04,1.480681e+04,3.914222e+04,147.255004,2688.223993,3489.078406,6.696759e-09,8.881533e-09,57388.499407,3.253071e-08,-1.907167e-08,7.593889e-09,9.738073e-09,False,1.231855e-08,9.945693e-09,False,-4.142626e-18,1.391829e-17,-8.617341e-18,2.143979e-17,2.393771e-17,7.779869e-17,-6.835623e-17,3.911214e-17,-1.698015e-17,-6.656052e-17,1.507869,0,0,131072
4,2449851475942276096,0.000459,-0.011376,5.978308e+04,4.750029e+04,6.862387e+04,142.485253,1799.160018,3502.530301,1.461376e-09,1.094439e-09,57388.499407,3.870431e-09,-1.609006e-08,1.784137e-09,1.147272e-09,False,5.773058e-11,1.771407e-09,False,4.012855e-19,-1.812061e-19,-2.576613e-19,1.386868e-19,-3.337862e-19,6.039006e-19,-6.034389e-19,5.569043e-19,-2.432351e-19,-8.279043e-19,0.554723,0,0,131072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,2448416132231612032,0.035143,-0.049295,1.530751e+05,5.062338e+04,3.428522e+05,224.576392,1988.310234,3080.030483,7.711772e-10,5.541608e-10,57388.499407,-2.916298e-08,-5.356730e-08,8.622686e-10,6.061082e-10,False,1.018524e-08,1.003278e-09,False,9.320426e-20,-5.398790e-20,-1.411602e-19,2.906244e-19,-9.850148e-20,1.265783e-19,-1.672269e-19,1.497548e-19,-1.258155e-20,-2.951270e-19,0.000000,0,0,131093
750,2448416136528007424,0.035323,-0.049236,4.434828e+06,2.563576e+06,7.224898e+06,1023.740133,3429.491947,4660.985585,1.422196e-10,1.158174e-10,57388.499407,1.039001e-07,1.203747e-07,1.889079e-10,1.201125e-10,False,1.399159e-08,1.794363e-10,False,3.955980e-21,2.024690e-21,-2.106096e-21,1.792190e-21,-4.914175e-21,6.978280e-21,-7.883634e-21,4.588280e-21,4.919064e-21,-8.179399e-21,0.180073,0,0,131093
751,2448416200954001024,0.035143,-0.049265,2.061542e+04,1.118043e+04,5.530383e+04,178.161583,2268.564542,4308.416798,5.675256e-09,4.670418e-09,57388.499407,-1.502303e-08,-3.231271e-08,5.979272e-09,5.463970e-09,False,8.545830e-09,7.394315e-09,False,3.778780e-18,8.222034e-18,-5.320911e-18,1.504281e-17,3.990825e-19,1.464144e-17,-1.553627e-17,1.097234e-17,-4.826500e-18,-2.304894e-17,2.946901,0,0,131093
752,2448427677103701376,0.033754,-0.049470,5.409050e+05,4.153565e+05,6.864771e+05,339.107277,2287.899279,3677.684947,3.612393e-10,2.937196e-10,57388.499407,3.634364e-08,-3.800179e-08,4.407886e-10,3.039638e-10,False,1.744359e-09,4.566746e-10,False,3.360614e-20,-7.868031e-21,-1.712596e-20,5.969909e-20,-2.0953

In [32]:
df.to_csv(filesource_out)

In [33]:
print(df.columns.tolist())

['id', 'ra', 'dec', 'phot_g_mean_flux', 'phot_bp_mean_flux', 'phot_rp_mean_flux', 'phot_g_mean_fluxErr', 'phot_bp_mean_fluxErr', 'phot_rp_mean_fluxErr', 'coord_raErr', 'coord_decErr', 'epoch', 'pm_ra', 'pm_dec', 'pm_raErr', 'pm_decErr', 'pm_flag', 'parallax', 'parallaxErr', 'parallax_flag', 'coord_ra_coord_dec_Cov', 'coord_ra_pm_ra_Cov', 'coord_ra_pm_dec_Cov', 'coord_ra_parallax_Cov', 'coord_dec_pm_ra_Cov', 'coord_dec_pm_dec_Cov', 'coord_dec_parallax_Cov', 'pm_ra_pm_dec_Cov', 'pm_ra_parallax_Cov', 'pm_dec_parallax_Cov', 'astrometric_excess_noise', 'tractID', 'patchID', 'skypix']


In [34]:
df['tract'].unique()

KeyError: 'tract'

In [ ]:
ref.to_simple()